In [37]:
import datetime
import json
import asyncio

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,49f65ad7-d8a2-4ab9-8ca0-ba777a2e0ea2
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jakejongejans_0000313927
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key


In [38]:
slice = fablib.get_slice(name="DYNAMOS-on-FABRIC");
nodes = slice.get_nodes();
network = slice.get_networks()[0];

In [39]:
def get_ip(node):
    interface = node.get_interface(network_name="NET1")
    return interface.get_ip_addr()

ips = [get_ip(node) for node in nodes];

print(ips);

['10.145.1.2', '10.145.1.3', '10.145.1.4']


In [40]:
inventory = (
    f"[kube_control_plane]\n"
    f"node1 ansible_host={ips[0]} ip={ips[0]} etcd_member_name=etcd1\n"
    f"\n"
    f"[etcd:children]\n"
    f"kube_control_plane\n"
    f"\n"
    f"[kube_node]\n"
    f"node2 ansible_host=10.145.5.3 ip=10.145.5.3\n"
    f"node3 ansible_host=10.145.5.4 ip=10.145.5.4\n"
)

for i, ip in enumerate(ips):
    inventory + f"node{i + 1} ansoble_host={ip} ip={ip}\n"

with open('kubespray/inventory.ini', 'w') as f:
    f.write(inventory)

In [46]:
nodes[0].upload_file(local_file_path="node_scripts/control_kubespray_setup.sh", remote_file_path="kubespray_setup.sh")
nodes[0].execute(f"chmod +x kubespray_setup.sh && ./kubespray_setup.sh");

nodes[0].upload_file(local_file_path="kubespray/inventory.ini", remote_file_path="kubespray/inventory/dynamos/inventory.ini")
nodes[0].upload_file(local_file_path="kubespray/ansible.cfg", remote_file_path="kubespray/ansible.cfg")

Hit:2 http://security.ubuntu.com/ubuntu noble-security InRelease
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Hit:1 https://prod-cdn.packages.k8s.io/repositories/isv:/kubernetes:/core:/stable:/v1.30/deb  InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:5 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Fetched 256 kB in 1s (228 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
python3 is already the newest version (3.12.3-0ubuntu2).
python3-pip is already the newest version (24.0+dfsg-1ubuntu1.1).
The following additional packages will be installed:
  python3-pip-whl python3-setuptools-whl python3.12-venv
The following NEW packages will be installed:
  python3-pip-whl python3-setuptools-whl python3-venv python3.12-venv
0 upgraded, 4 newly installed, 0 to remove and 4 not upgraded.
Need to get 2425 kB of archives.
After this operatio

<SFTPAttributes: [ size=756 uid=1000 gid=1000 mode=0o100664 atime=1744027911 mtime=1744027916 ]>